In [73]:
import pandas as pd
import numpy as np
import json
import os
import time
import openai
from tqdm import tqdm


In [6]:
openai.api_key='1f6c65a6a30b4a60ae2c377ba944d3a8'
openai.api_base='https://cairllm2.openai.azure.com/'
openai.api_type='azure'
openai.api_version='2023-07-01-preview'
#openai.azure_endpoint = "https://operationllm.openai.azure.com/"
engine="gpt-4"
azure_endpoint = "https://operationllm.openai.azure.com/"

In [7]:
anno_dir = "/home/lxj/dataset/Wales_anno_4604/annotation-upload"
anno_files = os.listdir(anno_dir)
flag = True
for file in anno_files:
    file_path = os.path.join(anno_dir,file)
    csv_file = pd.read_csv(file_path,sep=";")
    if flag:
        dataset = csv_file
        flag=False
    else:
        dataset = pd.concat([dataset,csv_file])
dataset = dataset.dropna(axis=0,how="all")
dataset = dataset.dropna(axis=1,how="all")
dataset = dataset[['video', 'folder', 'filename', 'stage', 'tool-1', 'action-1',
       'tissue-1', 'tool-2', 'action-2', 'tissue-2', 'tool-3', 'action-3',
       'tissue-3', 'tool-4', 'action-4', 'tissue-4' ]]

In [36]:
dataset

,video,folder,filename,stage,tool-1,action-1,tissue-1,tool-2,action-2,tissue-2,tool-3,action-3,tissue-3,tool-4,action-4,tissue-4,unique_id_1,unique_id_2
0,87.0,2021-03-13_100428,2021-03-13_100428_VID001_0_30.mp4,L,suction tube,manipulate,packing material,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-13_1004282021-03-13_100428_VID001_0_30,2021-03-13_100428_VID001_0_30
1,87.0,2021-03-13_100428,2021-03-13_100428_VID001_30_60.mp4,L,suction tube,manipulate,packing material,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-13_1004282021-03-13_100428_VID001_30_60,2021-03-13_100428_VID001_30_60
2,87.0,2021-03-13_100428,2021-03-13_100428_VID001_60_90.mp4,-1,suction tube,manipulate,tissue,freer,manipulate,tissue,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-13_1004282021-03-13_100428_VID001_60_90,2021-03-13_100428_VID001_60_90
3,87.0,2021-03-13_100428,2021-03-13_100428_VID001_120_150.mp4,L,freer,manipulate,tissue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-13_1004282021-03-13_100428_VID001_120_150,2021-03-13_100428_VID001_120_150
4,87.0,2021-03-13_100428,2021-03-13_100428_VID001_180_210.mp4,L,suction tube,manipulate,packing material,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-13_1004282021-03-13_100428_VID001_180_210,2021-03-13_100428_VID001_180_210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,33.0,20210709 hn21091970_092400,VID001_O_180_210.mp4,Z,rongeur,manipulate,packing material,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210709_hn21091970_092400VID001_O_180_210,VID001_O_180_210
270,33.0,20210709 hn21091970_092400,VID001_O_210_240.mp4,-1,rongeur,manipulate,packing material,freer,manipulate,tissue,NaN,NaN,NaN,NaN,NaN,NaN,20210709_hn21091970_092400VID001_O_210_240,VID001_O_210_240
271,33.0,20210709 hn21091970_092400,VID001_O_240_263.mp4,K,freer,manipulate,tissue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210709_hn21091970_092400VID001_O_240_263,VID001_O_240_263
272,33.0,20210709 hn21091970_092400,VID002_0_30.mp4,Z,rongeur,manipulate,tissue,rongeur,manipulate,packing material,NaN,NaN,NaN,NaN,NaN,NaN,20210709_hn21091970_092400VID002_0_30,VID002_0_30


In [75]:
def ask_gpt(messages, max_retries=35, temperature=0, top_p=0.9, max_tokens=512):
    if isinstance(messages, str):
        messages = [{"role": "user", "content": messages}]
    #print(messages)
    for i in range(max_retries):
        try:
            response = openai.ChatCompletion.create(
            engine="gpt-4", # engine = "deployment_name".
                messages=messages
)
            if os.getenv('DEBUG_PRINT'):
                print(response['choices'][0]['message']['content'])
            return response['choices'][0]['message']['content']
        except Exception as e:
            if type(e) in [openai.error.InvalidRequestError, KeyError]:
                print(type(e), e)
                return None
            print(type(e), e)
            time.sleep(2)
            continue    

In [96]:
dataset[dataset["unique_id_2"]=="VID001_D_510_540"]

,video,folder,filename,stage,tool-1,action-1,tissue-1,tool-2,action-2,tissue-2,tool-3,action-3,tissue-3,tool-4,action-4,tissue-4,unique_id_1,unique_id_2
76,58.0,20220121 hn22009508_,VID001_D_510_540.mp4,Z,driller,drill,bone,suction tube,suction,blood,NaN,NaN,NaN,NaN,NaN,NaN,20220121_hn22009508_VID001_D_510_540,VID001_D_510_540
72,46.0,20211029 hn21149741_092000,VID001_D_510_540.mp4,NaN,suction tube,suction,blood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20211029_hn21149741_092000VID001_D_510_540,VID001_D_510_540
64,40.0,20210903 HN21121139U__093255,VID001_D_510_540.mp4,NaN,suction tube,suction,blood,rongeur,remove,bone,NaN,NaN,NaN,NaN,NaN,NaN,20210903_HN21121139U__093255VID001_D_510_540,VID001_D_510_540
72,45.0,20211028 hn21149021_091645 ICG+,VID001_D_510_540.mp4,NaN,rongeur,remove,tissue,suction tube,suction,blood,NaN,NaN,NaN,NaN,NaN,NaN,20211028_hn21149021_091645_ICG+VID001_D_510_540,VID001_D_510_540
58,34.0,20210717 hn21096170_092019,VID001_D_510_540.mp4,-1,freer,manipulate,tissue,suction tube,suction,blood,rongeur,remove,bone,NaN,NaN,NaN,20210717_hn21096170_092019VID001_D_510_540,VID001_D_510_540
70,31.0,20210619 hn21082069_093022,VID001_D_510_540.mp4,NaN,suction tube,suction,blood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210619_hn21082069_093022VID001_D_510_540,VID001_D_510_540
71,32.0,20210703 hn21088951_093430,VID001_D_510_540.mp4,D,suction tube,remove,bone,rongeur,cut,tissue,NaN,NaN,NaN,NaN,NaN,NaN,20210703_hn21088951_093430VID001_D_510_540,VID001_D_510_540
68,39.0,20210821 hn21114591__094301,VID001_D_510_540.mp4,NaN,suction tube,suction,blood,rongeur,remove,bone,NaN,NaN,NaN,NaN,NaN,NaN,20210821_hn21114591__094301VID001_D_510_540,VID001_D_510_540
71,56.0,20210522 hn21068096P_093120,VID001_D_510_540.mp4,-1,freer,manipulate,tissue,rongeur,manipulate,tissue,NaN,NaN,NaN,NaN,NaN,NaN,20210522_hn21068096P_093120VID001_D_510_540,VID001_D_510_540
74,59.0,20220423 hn22045397_093745,VID001_D_510_540.mp4,D,suction tube,suction,blood,cup forcep,remove,bone,NaN,NaN,NaN,NaN,NaN,NaN,20220423_hn22045397_093745VID001_D_510_540,VID001_D_510_540


In [97]:
def get_caption(row):
    tuples = []
    for i in range(1,5):
        tool = "tool-"+str(i)
        action = "action-"+str(i)
        tissue = "tissue-"+str(i)
        single_tuple = []
        #print(row[tool])
        if not pd.isnull(row[tool].values):
            single_tuple.append(row[tool].values[0])
        if not pd.isnull(row[action].values):
            single_tuple.append(row[action].values[0])
        if not pd.isnull(row[tissue].values):
            single_tuple.append(row[tissue].values[0])
        if len(single_tuple)>0:
            tuples.append(single_tuple)
    if len(tuples)>0:
        prompt = "You are going to describe a Transsphenoidal pituitary adenoma resection surgery image in few words, the information you have is a tool's name, what kind of action the tool is doing, and what target the tool is operating on, the given tuple is {}. Please do not use any other information that did not mentioned in the tuple and describe in short. If receive more than one tuple, describe each tuple and concatenate normally. You can make the sentence more diverse with the same meaning."
        print(tuples)
        messages = prompt.format(str(tuples))
        return ask_gpt(messages)
    else:
        print("no tuple selected")
        return None

In [9]:
example_json_path = "/home/lxj/dataset/llava_plus_dataset/json_dataset/no_tool_origin_20000.json"
with open(example_json_path,"r") as f:
    example_json = json.load(f)
thought_list = []
for i in example_json:
    thought_list.append(i["conversations"][1]["thoughts"])

In [10]:
anno_file_path = dataset["folder"].astype(str)+dataset["filename"]

In [11]:
anno_file_paths = [i.split(".")[0] for i in anno_file_path]


In [12]:
anno_file_paths = [i.replace(" ","_") for i in anno_file_paths]



In [78]:
#anno_file_paths.sort()

In [13]:
img_folder = "/home/lxj/dataset/Wales_anno_4604/train/input"
imgs = os.listdir(img_folder)

In [119]:
#imgs = ["_".join(i.split(".")[0].split("_")[:-1]) for i in imgs]

In [14]:
anno_file_path_2 = dataset["filename"]
anno_file_paths_2 = [i.split(".")[0] for i in anno_file_path_2]
anno_file_paths_2 = [i.replace(" ","_") for i in anno_file_paths_2]


In [15]:
dataset["unique_id_1"] = anno_file_paths
dataset["unique_id_2"] = anno_file_paths_2

In [47]:
#用folder+filename组合查找img是否有annotation
count = 0
img_with_anno = []
for i in imgs:
    unique_id = "_".join(i.split(".")[0].split("_")[:-1])
    if unique_id in anno_file_paths:
        count+=1
        img_with_anno.append(i)
print(count)

802


In [48]:
#用filename查找img是否有annotation
count = 0
img_with_anno_2 = []
for i in imgs:
    unique_id = "_".join(i.split(".")[0].split("_")[:-1])
    if unique_id in anno_file_paths_2:
        count+=1
        img_with_anno_2.append(i)
print(count)

379


In [50]:
example_questions = [
    "Can you explain this image?",
    "Please explain this image",
    "Tell me about this image",
    "What is this image about?"
    "What is shown in this image?",
    "What is this image?",
    "Can you describe this image?",
    "What can you see in the image?"
    "Can you explain what this image depicts?",
    "What's the content of this picture?",
    "Provide a description of this image, please.",
    "What do you see in this image?",
    "Could you detail what's in this photo?",
    "What's illustrated in this photograph?",
    "Tell me about what this image shows.",
    "What is the subject of this image?",
    "How would you describe this picture?",
    "What's going on in this image?",
    "Can you give me details about this picture?",
    "What is featured in this photograph?",
    "Can you interpret this image for me?",
    "Describe what's pictured here.",
    "What does this image show?",
    "Could you give a brief overview of this image?",
    "What's captured in this photograph?",
    "Can you provide some context for this image?",
    "What are we looking at in this picture?",
    "Please provide a detailed description of the image.",
    "Elaborate on the details you observe in this image.",
    "Give a comprehensive description of everything shown in the image.",
    "Describe in detail the various elements present in the photograph.",
    "Offer a thorough explanation of what this image depicts.",
    "Examine the image closely and describe it in detail.",
    "Detail all the aspects of the image from foreground to background.",
    "Convey in depth the contents of this picture.",
    "Paint a vivid picture through words of what you see in this image.",
    "Break down the image into its components and describe each one in detail.",
    "Narrate the story that this image tells, focusing on specific details.",
    "Catalog every item and feature you can identify in the image.",
    "Provide an in-depth analysis of the visual elements in this photograph.",
    "Discuss the color, texture, and elements in the image with great detail.",
    "Give a blow-by-blow account of the scene depicted in this image.",
    "Craft a rich and detailed description of the image as if explaining to someone who cannot see it.",
    "Share a minute description of the image, including mood, tone, and style.",
    "Render a detailed narrative that captures the essence of the image.",
    "Express in vivid detail each component and its placement within the image.",
    "Conduct a detailed visual inspection of the image and report on your findings.",
    "Can you describe the image in detail, including colors, shapes, and textures?",
    "Could you provide a thorough description of what's happening in this picture?",
    "What detailed features can you identify in this photograph?",
    "Would you mind elaborating on the specific elements within this image?",
    "How would you detail the composition and subjects of this image?",
    "Can you give an in-depth description of the mood and atmosphere in this picture?",
    "What intricate details stand out to you in this image?",
    "Could you dissect the image and describe each component you see?",
    "In what detail can you describe the setting and characters depicted in this image?",
    "Can you capture the essence of this image through a detailed verbal depiction?",
    "What are the detailed visual elements that tell the story in this photograph?",
    "Could you provide a granular description of the textures and patterns in this image?",
    "How would you describe the interplay of light and shadow in this image in detail?",
    "Can you describe in fine detail the expressions and emotions captured in this image?",
    "What specific details in this image give context to its overall theme?",
    "Can you walk me through the finer details that might be overlooked in this image?",
    "How would you describe the stylistic details of the objects or subjects in this photograph?",
    "Could you detail the foreground, middleground, and background of this image?",
    "What elements of this image are crucial for a detailed understanding of its message?",
    "Can you analyze and describe the intricate details that contribute to the overall impact of this image?",
    "Are you able to depict the contents of this surgical image in words?",
    "Could you provide an explanation of what's shown in this surgical photo?",
    "Is it possible for you to detail what this surgical image is displaying?",
    "May you offer a description of what is captured in this surgical picture?",
    "Would you mind giving a narrative of this image from the surgery?",
    "Can you illustrate what's going on in this image of the surgical procedure?",
    "Are you capable of outlining the features in this surgical operation snapshot?",
    "Could you verbalize the specifics seen in this image of a surgical setting?",
    "Might you convey what is represented in this surgical image?",
    "Can you present a clear depiction of the elements in this surgical photo?",
    "Is it possible for you to explicate the scene shown in this surgical image?",
    "Would you be able to offer a visual description of this surgery-related image?",
    "Could you give your interpretation of what this surgical image shows?",
    "Can you provide a comprehensive description of what is pictured here in this surgery image?",
    "Is it possible for you to articulate the details of this surgical photograph?",
    "May you explain the scene depicted in this image from a surgical environment?",
    "Would you be able to delineate the components of this surgical image?",
    "Can you narrate the activities shown in this image from a surgical procedure?",
    "Are you equipped to describe the specifics of this surgical technique as shown in the image?",
    "Might you provide a detailed account of the surgical scene in this image?"

]

In [77]:
answer_list = []
img_folder = "/home/lxj/dataset/Wales_anno_4604/train/input"
for img in tqdm(img_with_anno):
    question_promt = np.random.choice(example_questions)
    thought = np.random.choice(thought_list)
    t_dict = {}
    unique_id = "_".join(img.split(".")[0].split("_")[:-1])
    t_dict["id"] = unique_id
    t_dict["image"] = img
    t_dict["data_source"] = "wales"
    t_dict["unique_id"] = unique_id
    selected_row = dataset[dataset["unique_id_1"]==unique_id]
    caption = get_caption(selected_row)
    print(caption)
    t_dict["conversations"] = [
        {
        "from": "human",
        "value": "<image>\n{q}".format(q=question_promt)
      },
      {
        "from": "gpt",
        "value": caption,
        "thoughts": thought,
        "actions": []
      },
    ]
    answer_list.append(t_dict)


  0%|          | 0/802 [00:00<?, ?it/s]

[['rongeur', 'manipulate', 'tissue']]


  0%|          | 1/802 [00:01<22:32,  1.69s/it]

The image depicts the manipulation of tissue with a rongeur during a Transsphenoidal pituitary adenoma resection surgery.
[['suction tube', 'suction', 'tumor'], ['bipolar electrode', 'cauterize', 'tissue']]


  0%|          | 2/802 [00:04<31:04,  2.33s/it]

...

The image depicts an ultrasound probe being used to sense a vessel during a Transsphenoidal pituitary adenoma resection surgery.
[['scissor', 'cut', 'tissue'], ['suction tube', 'suction', 'blood']]


100%|█████████▉| 801/802 [32:26<00:02,  2.36s/it]

The image depicts a scissor being used to cut tissue during a Transsphenoidal pituitary adenoma resection surgery. Furthermore, a suction tube is seen handling the removal of blood.
[['monopolar electrode', 'manipulate', 'tissue'], ['scalpel', 'cut', 'tumor']]


100%|██████████| 802/802 [32:31<00:00,  2.43s/it]

The image depicts a monopolar electrode being used to manipulate the tissue during a Transsphenoidal pituitary adenoma resection surgery. In another scene of the procedure, a scalpel can be observed making a cut into the tumor.


In [ ]:
#be careful, some will return null due to network error in openai

In [78]:
with open("wales_caption.json","w") as f:
    f.write(json.dumps(answer_list,indent=2))

In [92]:
answer_list_2 = []
img_folder = "/home/lxj/dataset/Wales_anno_4604/train/input"
for img in tqdm(img_with_anno_2):
    question_promt = np.random.choice(example_questions)
    thought = np.random.choice(thought_list)
    t_dict = {}
    unique_id = "_".join(img.split(".")[0].split("_")[:-1])
    t_dict["id"] = unique_id
    t_dict["image"] = img
    t_dict["data_source"] = "wales"
    t_dict["unique_id"] = unique_id
    selected_row = dataset[dataset["unique_id_2"]==unique_id]
    caption = get_caption(selected_row)
    print(caption)
    t_dict["conversations"] = [
        {
        "from": "human",
        "value": "<image>\n{q}".format(q=question_promt)
      },
      {
        "from": "gpt",
        "value": caption,
        "thoughts": thought,
        "actions": []
      },
    ]
    answer_list_2.append(t_dict)

  0%|          | 0/379 [00:00<?, ?it/s]

[['curette', 'scoop', 'tumor']]


  0%|          | 1/379 [00:01<11:49,  1.88s/it]

The curette is being used to scoop out the tumor in a Transsphenoidal pituitary adenoma resection surgery.
[['suction tube', 'suction', 'blood'], ['rongeur', 'remove', 'bone']]


  1%|          | 2/379 [00:04<14:25,  2.29s/it]

...

  2%|▏         | 6/379 [00:12<14:15,  2.29s/it]

A suction tube is being used to suction a tumor in a Transsphenoidal pituitary adenoma resection surgery.
[['suction tube', 'suction', 'blood'], ['cutting forcep', 'remove', 'bone']]


  2%|▏         | 7/379 [00:20<17:44,  2.86s/it]

The image showcases a suction tube being used to aspirate blood during a transsphenoidal pituitary adenoma resection surgery. Additionally, the image depicts a cutting forcep being utilized to remove bone.


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
with open("wales_caption_2.json","w") as f:
    f.write(json.dumps(answer_list_2,indent=2))

In [98]:
import shutil

In [100]:
for img in img_with_anno:
    img_path = os.path.join(img_folder,img)
    des_path = os.path.join("/home/lxj/dataset/wales_caption_img",img)
    shutil.copyfile(img_path,des_path)

In [ ]:
#将掩码转换成boundingbox
from PIL import Image
import numpy as np

def extract_bounding_boxes(mask_path, num_classes):
    # 读取掩码图像
    mask = Image.open(mask_path)
    mask_array = np.array(mask)

    # 确保掩码有三个颜色通道
    if mask_array.shape[2] != 3:
        raise ValueError("Mask does not have 3 color channels.")

    # 初始化每个类别的边界值
    bounding_boxes = {cls: [np.inf, np.inf, -np.inf, -np.inf] for cls in range(num_classes)}

    # 获取图像尺寸
    height, width, _ = mask_array.shape

    # 遍历每个像素
    for y in range(height):
        for x in range(width):
            # 获取像素值（类别）
            class_idx = mask_array[y, x, 0]  # 假设类别信息存储在第一个通道

            # 更新类别的 bounding box
            if class_idx in bounding_boxes:
                xmin, ymin, xmax, ymax = bounding_boxes[class_idx]
                xmin = min(xmin, x)
                ymin = min(ymin, y)
                xmax = max(xmax, x)
                ymax = max(ymax, y)
                bounding_boxes[class_idx] = [xmin, ymin, xmax, ymax]

    # 清除无效的 bounding box（即那些没有找到像素的类别）
    for cls, bbox in list(bounding_boxes.items()):
        if bbox[0] == np.inf:
            del bounding_boxes[cls]

    # 返回所有有效的 bounding boxes
    return bounding_boxes

# 使用函数提取 bounding boxes
mask_path = 'path_to_your_mask.png'
num_classes = 3  # 根据你的分割类别数量这里可能需要更改
bounding_boxes = extract_bounding_boxes(mask_path, num_classes)

# 打印结果
for cls, bbox in bounding_boxes.items():
    print(f"Class {cls}: Bounding Box: {bbox}")